In [2]:
import pandas as pd
import numpy as np
import seaborn as sns


In [3]:
sales = pd.read_csv(r"../aws_retail_ds_exp/Sales Data.csv")
inventory = pd.read_csv(r"../aws_retail_ds_exp/Inventory Data.csv")
pricing = pd.read_csv(r"../aws_retail_ds_exp/Competitor Pricing Data.csv")
behavior = pd.read_csv(r"../aws_retail_ds_exp/Daily Customer Behavior.csv")

In [4]:
inventory['Date'] = pd.to_datetime(inventory['Date'], format = "%Y-%m-%d")
sales['TransactionDate'] = pd.to_datetime(sales['TransactionDate'], format="%d-%m-%Y")
behavior['Date'] = pd.to_datetime(behavior['Date'], format = "%d-%m-%Y")
pricing['Date'] = pd.to_datetime(pricing['Date'], format = "%d-%m-%Y")

In [5]:
# Pivot competitor pricing data so that each brand has its own column
competitor_pivot_data = pricing.pivot(index="Date", columns="Brand", values=["MRP", "DiscountRate", "BasePrice", "FinalPrice"])

# Flatten the MultiIndex columns for better readability
competitor_pivot_data.columns = ['_'.join(col).strip() for col in competitor_pivot_data.columns]

# Reset index to turn Date back into a column
competitor_pivot_data.reset_index(inplace=True)

# Display the final pivoted competitor data
competitor_pivot_data.head()

,Date,MRP_Nirma,MRP_Surf Excel,DiscountRate_Nirma,DiscountRate_Surf Excel,BasePrice_Nirma,BasePrice_Surf Excel,FinalPrice_Nirma,FinalPrice_Surf Excel
0,2021-01-01,90.0,110.0,0.00,0.00,84.67,107.24,84.67,107.24
1,2021-01-02,90.0,110.0,0.00,0.07,81.16,107.90,81.16,100.35
2,2021-01-03,90.0,110.0,0.00,0.00,79.33,105.83,79.33,105.83
3,2021-01-04,90.0,110.0,0.00,0.00,76.72,101.36,76.72,101.36
4,2021-01-05,90.0,110.0,0.24,0.00,70.80,107.07,60.00,107.07


In [6]:
# Define the aggregation strategy
inventory_aggregation_dict = {
    'StockStart': 'sum',
    'Demand': 'sum',
    'DemandFulfilled': 'sum',
    'Backorders': 'sum',
    'StockEnd': 'min',  # Min for StockEnd, to capture lowest stock levels
    'ReorderPoint': 'max',  # Max for ReorderPoint, as it's a threshold
    'OrderPlaced': 'sum',
    'OrderQty': 'sum',
    'LeadTimeFloat': 'mean',  # Weighted average for LeadTimeFloat
    'SafetyStock': 'mean'  # Weighted average for SafetyStock
}

# Perform the aggregation
inventory_aggregated_data = inventory.groupby('Date').agg(inventory_aggregation_dict).reset_index()

In [7]:
inventory_aggregated_data.head() 

,Date,StockStart,Demand,DemandFulfilled,Backorders,StockEnd,ReorderPoint,OrderPlaced,OrderQty,LeadTimeFloat,SafetyStock
0,2021-01-01,15662,1831,1831,0,67,1023,6,2604,3.8755,178.45
1,2021-01-02,14162,2097,2097,0,37,1023,7,4260,3.8755,178.45
2,2021-01-03,12691,1837,1814,23,0,1023,10,6742,3.8755,178.45
3,2021-01-04,13081,1588,1588,0,48,1023,10,7040,3.8755,178.45
4,2021-01-05,13358,1912,1912,0,3,1023,8,7227,3.8755,178.45


In [8]:
merged_data = (
    sales.rename(columns={"TransactionDate": "Date"})
    .merge(inventory_aggregated_data, on="Date", how="left")  # Left join for inventory
    .merge(behavior, on="Date", how="inner")  # Inner join for customer data (only dates with customer behavior)
    .merge(competitor_pivot_data, on="Date", how="left")  # Left join for competitor data
)

In [9]:
merged_data.shape

(365, 30)

In [10]:
# 1. Price Elasticity (change in units sold and price)
merged_data['PriceChangePct'] = merged_data['SellingPrice'].pct_change() * 100
merged_data['UnitsSoldChangePct'] = merged_data['UnitsSold'].pct_change() * 100
merged_data['PriceElasticity'] = merged_data['UnitsSoldChangePct'] / merged_data['PriceChangePct']

merged_data['PriceChangePct'].fillna(0)
merged_data['UnitsSoldChangePct'].fillna(0)
merged_data['PriceElasticity'].fillna(0)

# 2. Price Difference from Competitors
merged_data['PriceDiff_Nirma'] = merged_data['SellingPrice'] - merged_data['FinalPrice_Nirma']
merged_data['PriceDiff_SurfExcel'] = merged_data['SellingPrice'] - merged_data['FinalPrice_Surf Excel']

# 3. Discount Difference
merged_data['DiscountDiff_Nirma'] = merged_data['DiscountRate_Nirma'] - (merged_data['SellingPrice'] / merged_data['MRP'])
merged_data['DiscountDiff_SurfExcel'] = merged_data['DiscountRate_Surf Excel'] - (merged_data['SellingPrice'] / merged_data['MRP'])

# 4. Log-transformed price (already provided as 'LogSellingPrice')
# It's already present, no need to recompute, but ensure it's kept in the dataset.

# 5. Customer Engagement Metrics
# You may want to aggregate these over time to observe trends and customer behavior changes:
# merged_data['AvgCTR'] = merged_data['CTR'].rolling(window=7).mean()
# merged_data['AvgBounceRate'] = merged_data['BounceRate'].rolling(window=7).mean()
# merged_data['AvgSessionDuration'] = merged_data['AvgSessionDuration_sec'].rolling(window=7).mean()

# 6. Create additional features based on trends and rolling statistics:
# merged_data['UnitsSold_7DayAvg'] = merged_data['UnitsSold'].rolling(window=7).mean()
# merged_data['Demand_7DayAvg'] = merged_data['Demand'].rolling(window=7).mean()

# 7. Create a feature to capture whether a product was on discount during a given period
merged_data['OnDiscount'] = np.where(merged_data['DiscountRate_Nirma'] > 0.1, 1, 0)


# Display the newly created features
merged_data[['Date', 'UnitsSold', 'PriceElasticity', 'PriceDiff_Nirma', 'PriceDiff_SurfExcel', 'DiscountDiff_Nirma', 'DiscountDiff_SurfExcel', 'OnDiscount']].head()


,Date,UnitsSold,PriceElasticity,PriceDiff_Nirma,PriceDiff_SurfExcel,DiscountDiff_Nirma,DiscountDiff_SurfExcel,OnDiscount
0,2021-01-01,12,NaN,13.28,-9.29,-0.989394,-0.989394,0
1,2021-01-02,10,1.269440,3.93,-15.26,-0.859495,-0.789495,0
2,2021-01-03,4,-6.511990,13.60,-12.90,-0.938687,-0.938687,0
3,2021-01-04,7,-25.909851,13.52,-11.12,-0.911515,-0.911515,0
4,2021-01-05,12,26.525573,32.67,-14.40,-0.696061,-0.936061,1


In [11]:
merged_data['PriceChangePct'].fillna(0, inplace=True)
merged_data['UnitsSoldChangePct'].fillna(0, inplace=True)
merged_data['PriceElasticity'].fillna(0, inplace=True)

In [12]:
merged_data[merged_data.select_dtypes(include=['float']).columns] = merged_data.select_dtypes(include=['float']).round(2)

In [13]:
merged_data.keys()

Index(['Date', 'MRP', 'NoPromoPrice', 'SellingPrice', 'UnitsSold',
       'StockStart', 'Demand', 'DemandFulfilled', 'Backorders', 'StockEnd',
       'ReorderPoint', 'OrderPlaced', 'OrderQty', 'LeadTimeFloat',
       'SafetyStock', 'CTR', 'AbandonedCartRate', 'BounceRate',
       'FunnelDrop_ViewToCart', 'FunnelDrop_CartToCheckout',
       'ReturningVisitorRatio', 'AvgSessionDuration_sec', 'MRP_Nirma',
       'MRP_Surf Excel', 'DiscountRate_Nirma', 'DiscountRate_Surf Excel',
       'BasePrice_Nirma', 'BasePrice_Surf Excel', 'FinalPrice_Nirma',
       'FinalPrice_Surf Excel', 'PriceChangePct', 'UnitsSoldChangePct',
       'PriceElasticity', 'PriceDiff_Nirma', 'PriceDiff_SurfExcel',
       'DiscountDiff_Nirma', 'DiscountDiff_SurfExcel', 'OnDiscount'],
      dtype='object')

In [14]:
import numpy as np

# Define threshold for large values
threshold = 1e308

# Replace infinite values with NaN
merged_data.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN (which were originally inf values)
merged_data.dropna(inplace=True)

# Select only float columns
float_cols = merged_data.select_dtypes(include=[np.number]).columns

# Drop rows where any float column has values greater than the threshold
merged_data = merged_data[(merged_data[float_cols] < threshold).all(axis=1)]

# Reset index after dropping rows
merged_data.reset_index(drop=True, inplace=True)


In [15]:
min_float64 = np.finfo(np.float64).min
max_float64 = np.finfo(np.float64).max

In [16]:
# Check if 'sellingprice' is within the float64 range
merged_data['sellingprice_within_range'] = merged_data['SellingPrice'].between(min_float64, max_float64)

# Check if 'date' is within the float64 range
# merged_data['date_within_range'] = merged_data['Date'].between(min_float64, max_float64)

In [17]:
from sklearn.preprocessing import LabelEncoder
# Encode 'sellingprice_within_range' column
label_encoder = LabelEncoder()
merged_data['sellingprice_within_range'] = label_encoder.fit_transform(merged_data['sellingprice_within_range'])

In [18]:
merged_data.isnull().sum()
merged_data.duplicated
merged_data.info()
merged_data.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364 entries, 0 to 363
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Date                       364 non-null    datetime64[ns]
 1   MRP                        364 non-null    float64       
 2   NoPromoPrice               364 non-null    float64       
 3   SellingPrice               364 non-null    float64       
 4   UnitsSold                  364 non-null    int64         
 5   StockStart                 364 non-null    int64         
 6   Demand                     364 non-null    int64         
 7   DemandFulfilled            364 non-null    int64         
 8   Backorders                 364 non-null    int64         
 9   StockEnd                   364 non-null    int64         
 10  ReorderPoint               364 non-null    int64         
 11  OrderPlaced                364 non-null    int64         
 12  OrderQty

,Date,MRP,NoPromoPrice,SellingPrice,UnitsSold,StockStart,Demand,DemandFulfilled,Backorders,StockEnd,...,FinalPrice_Surf Excel,PriceChangePct,UnitsSoldChangePct,PriceElasticity,PriceDiff_Nirma,PriceDiff_SurfExcel,DiscountDiff_Nirma,DiscountDiff_SurfExcel,OnDiscount,sellingprice_within_range
count,364,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,...,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.0
mean,2021-07-01 18:47:28.351648512,101.488599,89.713297,88.573626,8.640110,41448.266484,1948.736264,1940.568681,8.167582,98.741758,...,105.273104,0.325824,48.992390,-1.682335,10.382967,-16.699478,-0.849011,-0.866978,0.118132,0.0
min,2021-01-01 00:00:00,99.000000,71.740000,66.440000,0.000000,12691.000000,1356.000000,1356.000000,0.000000,0.000000,...,86.530000,-27.050000,-100.000000,-2583.900000,-17.260000,-36.710000,-1.000000,-1.000000,0.000000,0.0
25%,2021-04-01 18:00:00,99.000000,86.267500,85.082500,5.000000,37600.000000,1747.000000,1738.000000,0.000000,37.750000,...,103.685000,-6.287500,-44.440000,-8.732500,4.560000,-21.590000,-0.900000,-0.910000,0.000000,0.0
50%,2021-07-01 12:00:00,103.950000,89.715000,88.995000,8.000000,41443.000000,1929.500000,1913.000000,0.000000,94.000000,...,106.335000,-0.175000,0.000000,-0.000000,10.060000,-16.710000,-0.860000,-0.870000,0.000000,0.0
75%,2021-10-01 06:00:00,103.950000,93.212500,92.607500,12.000000,46003.250000,2115.500000,2115.500000,0.000000,153.000000,...,107.960000,6.180000,66.670000,8.882500,15.757500,-12.355000,-0.810000,-0.830000,0.000000,0.0
max,2021-12-31 00:00:00,103.950000,103.840000,103.840000,34.000000,60621.000000,2808.000000,2808.000000,216.000000,329.000000,...,109.990000,35.840000,1400.000000,2272.000000,37.280000,13.770000,-0.510000,-0.620000,1.000000,0.0
std,NaN,2.478369,5.101517,6.020082,4.817282,7915.534638,275.647071,274.623024,27.939071,74.172788,...,3.955099,9.487634,167.791611,228.838651,9.325165,7.274557,0.082923,0.065666,0.323208,0.0


In [21]:
merged_data.to_csv(r"../aws_retail_ds_exp/training_data.csv")